In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Installed Compat ─────────── v1.1.0
 Installed ZipFile ────────── v0.7.0
 Installed Missings ───────── v0.3.0
 Installed Media ──────────── v0.4.1
 Installed CodecZlib ──────── v0.5.0
 Installed DataStructures ─── v0.12.0
 Installed SpecialFunctions ─ v0.7.0
 Installed ForwardDiff ────── v0.9.0
 Installed NNlib ──────────── v0.4.2
 Installed Flux ───────────── v0.6.7
 Installed ColorTypes ─────── v0.7.4
 Installed BinaryProvider ─── v0.4.2
 Installed StaticArrays ───── v0.8.3
 Installed Colors ─────────── v0.9.4
  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/KvXoO/deps/build.log`
  Building CodecZlib ───────→ `~/.julia/packages/CodecZlib/wwgbh/deps/build.log`


Based on https://arxiv.org/abs/1409.0473

In [2]:
using Flux: flip, crossentropy, reset!, throttle

include("0-data.jl")

Nin = length(alphabet)
Nh = 30 # size of hidden layer

┌ Info: Downloading CMUDict dataset
└ @ Flux.Data.CMUDict /home/dhairyagandhi96/.julia/packages/Flux/UHjNa/src/data/cmudict.jl:17
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   441  100   441    0     0    930      0 --:--:-- --:--:-- --:--:--   928
100 3629k  100 3629k    0     0  4295k      0 --:--:-- --:--:-- --:--:-- 4295k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   455  100   455    0     0   2049      0 --:--:-- --:--:-- --:--:--  2049
100   382  100   382    0     0   1140      0 --:--:-- --:--:-- --:--:--  1140
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   457  100   457    0     0   2115      0 --:--:-- --:--:-- --:--:--  2105
1

30

A recurrent model which takes a token and returns a context-dependent
annotation.

In [3]:
forward  = LSTM(Nin, Nh÷2)
backward = LSTM(Nin, Nh÷2)
encode(tokens) = vcat.(forward.(tokens), flip(backward, tokens))

alignnet = Dense(2Nh, 1)
align(s, t) = alignnet(vcat(t, s .* trues(1, size(t, 2))))

align (generic function with 1 method)

A recurrent model which takes a sequence of annotations, attends, and returns
a predicted output token.

In [4]:
recur   = LSTM(Nh+length(phones), Nh)
toalpha = Dense(Nh, length(phones))

function asoftmax(xs)
  xs = [exp.(x) for x in xs]
  s = sum(xs)
  return [x ./ s for x in xs]
end

function decode1(tokens, phone)
  weights = asoftmax([align(recur.state[2], t) for t in tokens])
  context = sum(map((a, b) -> a .* b, weights, tokens))
  y = recur(vcat(float(phone), context))
  return softmax(toalpha(y))
end

decode(tokens, phones) = [decode1(tokens, phone) for phone in phones]

decode (generic function with 1 method)

The full model

In [5]:
state = (forward, backward, alignnet, recur, toalpha)

function model(x, y)
  ŷ = decode(encode(x), y)
  reset!(state)
  return ŷ
end

loss(x, yo, y) = sum(crossentropy.(model(x, yo), y))

evalcb = () -> @show loss(data[500]...)
opt = ADAM(params(state))

Flux.train!(loss, data, opt, cb = throttle(evalcb, 10))

loss(data[500]...) = 31.196248911316175 (tracked)
loss(data[500]...) = 18.972686562093664 (tracked)
loss(data[500]...) = 17.24633769361914 (tracked)
loss(data[500]...) = 15.78869506280649 (tracked)
loss(data[500]...) = 14.450438156943573 (tracked)
loss(data[500]...) = 14.519718743800802 (tracked)
loss(data[500]...) = 13.432460058485816 (tracked)
loss(data[500]...) = 12.655016075722912 (tracked)
loss(data[500]...) = 12.993700013880023 (tracked)
loss(data[500]...) = 12.513409757003476 (tracked)
loss(data[500]...) = 13.116094984540927 (tracked)
loss(data[500]...) = 13.25861052566218 (tracked)
loss(data[500]...) = 13.238037142798877 (tracked)
loss(data[500]...) = 13.906508968288195 (tracked)
loss(data[500]...) = 14.279464243950542 (tracked)
loss(data[500]...) = 15.443783676540672 (tracked)
loss(data[500]...) = 14.768633821443984 (tracked)
loss(data[500]...) = 15.163692827457256 (tracked)
loss(data[500]...) = 16.34684358955311 (tracked)
loss(data[500]...) = 18.429544301313356 (tracked)


Prediction

In [6]:
using StatsBase: wsample

function predict(s)
  ts = encode(tokenise(s, alphabet))
  ps = Any[:start]
  for i = 1:50
    dist = decode1(ts, onehot(ps[end], phones))
    next = wsample(phones, vec(Tracker.data(dist)))
    next == :end && break
    push!(ps, next)
  end
  return ps[2:end]
end

predict("PHYLOGENY")

15-element Array{Any,1}:
 :OW1
 :L  
 :AH0
 :JH 
 :AE1
 :N  
 :T  
 :R  
 :B  
 :Y  
 :EY2
 :L  
 :AH0
 :D  
 :IY0